# Importe

In [36]:
import os
import pandas as pd
import requests
import unicodedata
import re
from bs4 import BeautifulSoup
import string
from itertools import permutations
from tqdm import tqdm
import datetime

# Vorbereitungen
Zunächst wird eine Liste von Subdomains der Uni Bayreuth auf ihre Verfügbarkeit gefiltert.

In [37]:
def get_all_subdomains(path):
    """Liest alle Subdomains von www.uni-bayreuth.de ein."""
    subdomains = []
    f = open(path + 'subdomains.txt', 'r')
    line = f.readline()
    while line:
        subdomains.append(line.rstrip())
        line = f.readline()
    f.close()
    return subdomains

In [38]:
def check_subdomains(subdomains):
    """Überprüft für eine Liste von subdomains ob sie
    existieren und returned die, die es tun."""
    valid_subdomains = []
    for url in tqdm(subdomains):
        news_url = 'https://' + url
        try:
            page = requests.get(news_url)
        except:
            news_url = 'http://' + url
            try:
                page = requests.get(news_url)
            except:
                continue
        if page:
            valid_subdomains.append(news_url)
    return list(set(valid_subdomains))

In [ ]:
subdomains = get_all_subdomains('/home/ec2-user/data/')

In [ ]:
valid_subdomains = check_subdomains(subdomains)
valid_subdomains.extend(['https://www.mcii.uni-bayreuth.de/en/team/team-greiner/index.html',
              'https://www.mcii.uni-bayreuth.de/en/team/team-agarwal/index.html',
              'https://www.pes.uni-bayreuth.de/en/professorship/team/index.html'])

# URLs von Team-Seiten sammeln
Für die zuvor ermittelten Subdomains wird nun geprüft ob sie eine Team-Seite besitzen und falls ja, so wird diese gespeichert.

In [17]:
def get_team_url(subdomains):
    """Erstellt eine Liste mit den Team-Seiten aller 
    Urls in der Liste subdomains."""
    team_urls = []
    for url in subdomains:
        endings = ['team/index.html', 'people/index.html']
        for ending in endings:
            complete = url + ending
            page = requests.get(complete)
            if page:
                team_urls.append(complete)
                break
    return team_urls


In [18]:
team_urls = get_team_url(valid_subdomains)

# Team Seiten Scrapen
Die zuvor ermittelten Team-Seiten werden nun gescraped. Dabei werden relevante Informationen wie z.B. E-Mail oder Telefonnummer extrahiert und im Knowledge-Graph Format (target --> relation --> source) abgespeichert.

In [19]:
def get_team_info_in_kg_format(team_url):
    """Extrahiert die Informationen einer Lehrstuhlseite und speichert diese
    korrekten Format für den Knowledge-Graph ab."""
    page = requests.get(team_url)
    if not page:
        return None
    soup = BeautifulSoup(page.content, 'html.parser')
    kg_global = []
    lehrstuhl = get_chair_name(soup)
    team_members = get_team_member_info(soup)
    for member in team_members:
        for i in range(1,len(member)):
            name = replace_title(member[0])
            if i < 2:
                function = [name, member[1], lehrstuhl]
            else:
                temp = member[i].split()
                function = [name, temp[0], ' '.join(temp[1:])]
            function_clean = []
            for test in function:
                clean = test.strip()
                clean = clean.strip(string.punctuation)
                clean = clean.strip()
                function_clean.append(clean)
            for test in function_clean:
                if (test == '') or (len(test) < 3):
                    continue
            kg_global.append(function_clean)
    return kg_global

def get_chair_name(soup):
    """Extrahiert den Namen des Lehrstuhls aus HTML-Texten."""
    lehrstuhl = str(soup.find_all('h3')[0])
    lehrstuhl = re.sub('<.*?>', '', lehrstuhl)
    lehrstuhl = lehrstuhl.split('–')[0].strip()
    lehrstuhl = lehrstuhl.replace('&amp;', '&')
    return lehrstuhl

def get_team_member_info(soup):
    """Extrahiert für alle Mitarbeiter einer Lehrstuhlseite die
    veröffentlichten Informationen."""
    member_section = soup.find_all('section', class_ = 'fse-block fse-dropzone text full')
    all_members = []
    for member in member_section:
        member_info = member.find_all('p')
        member_info = [str(x) for x in member_info]
        member_info = ''.join(member_info)
        member_info = member_info.replace('</p>', ' *_* ')
        member_info = member_info.replace('</a>', ' *_* ')
        member_info = member_info.replace('\u200b', '')
        member_info = re.sub('<br.*?>', ' *_* ', member_info)
        team_member_clean = []
        for info in member_info.split('*_*'):
            team_member_clean.append((re.sub('<.*?>', '', info)).strip())
        team_member_clean = [unicodedata.normalize('NFKD', x) for x in team_member_clean if x != '']
        all_members.append(team_member_clean)
    return all_members



def replace_title(name):
    """Entfernt typische Titel im Namen eines Akademikers."""
    titles = ['Univ.', 'Prof.', 'Dr.', 'Professor', 'Doktor', 'Ing.', '-Ing.', 'rer.', 'PD', 'M.Sc.',
              'M.A.', 'M.Eng.', 'LL.M.', 'M.F.A.', 'M.Mus.', 'M.Ed.', 'B.A.', 'BBA', 'FH', 
              'B.Sc.', 'LL.B.', 'B.Ed.', 'B.Eng.', 'B.F.A.', 'B.Mus.', 'B.M.A.', 'Dipl.',
              'M.A', 'LL.M', 'LL.B', 'M.Sc', 'B.A', 'B.Sc', 'StB', 'Diplom-Jurist', 'Ass.',
              'jur.', 'ass.', 'Jur.', 'Wirtschaftsjurist', 'Wirtschaftsjuristin', 'SpOec',
              'Jun.', 'Mag.', 'nat.', 'M. Sc.', 'Wirtsch.', 'Kfm.' 'RA', 'LLP']
    for title in titles:
        name = name.replace(title, '')
        name = name.strip(string.punctuation)
        name = name.strip()
    return name

In [20]:
data_team = []
for url in tqdm(team_urls):
    try:
        team_info = get_team_info_in_kg_format(url)
    except Exception as e:
        continue
    if team_info:
        data_team.extend(team_info)

100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [00:32<00:00,  5.54it/s]


In [21]:
team_df = pd.DataFrame(data_team, columns = ['source', 'relation', 'target'])

# CampusOnline Vorlesungsseiten Scrapen
Nun werden für alle Vorlesungen im Sommersemester 2020 die relevanten Informationen wie z.B. Vorlesungsnummer oder Dozent extrahiert

In [22]:
faculty_numbers = ['14093', '14094', '14095', '14096', '14097', '14098', '15794']

In [23]:
lecture_links = []
for faculty in faculty_numbers:
    count = 1
    while True:
        base_url = 'https://campusonline.uni-bayreuth.de/ubto/wbLVAngebot.cbReloadLVOfferTable?pOrgNr=' + faculty + '&pPersonNr=&pSjNr=1709&pStpLvTypNr=-1&pPageNr=' + str(count)+ '&pSort=6%3B5&pFilter=null%3Ff_4_1%3DS&pGroup=K'
        page = requests.get(base_url)
        soup = BeautifulSoup(page.content, 'lxml')
        row = soup.find_all('div', class_="L")
        if len(row) == 0:
            break
        for i in row:
            a_tags = i.find_all('a')
            for a in a_tags:
                link = 'https://campusonline.uni-bayreuth.de/ubto/' + a['href']
                lecture_links.append(link)
        count += 1

In [24]:
kg_data = []
for lecture in tqdm(lecture_links):
    try:
        page = requests.get(lecture)
        soup = BeautifulSoup(page.content, 'html.parser')
        name = soup.find_all('span', class_ = 'bold')[1].text
        name = ' '.join(name.split())
        name = unicodedata.normalize('NFKD', name)
        number = soup.find_all('span', class_ = 'bold')[2].text
        number = unicodedata.normalize('NFKD', number)
        lecture_type = soup.find_all('span', class_ = 'bold')[3].text
        lecture_type = unicodedata.normalize('NFKD', lecture_type)
        dozent = soup.find_all('a', attrs = {'target' : 'SVI'})[0].text
        dozent = dozent.split(',')
        dozent.reverse()
        dozent = ' '.join(dozent).strip()
        dozent = unicodedata.normalize('NFKD', dozent)
        chair = soup.find_all('a', attrs = {'target' : 'tugonline'})[0].text
        chair = re.sub('\([^)]*\)$', '', chair).strip()
        chair = unicodedata.normalize('NFKD', chair)
        kg_data.append([name, 'Vorlesungsnummer', number])
        kg_data.append([name, 'Veranstaltungsart', lecture_type])
        kg_data.append([name, 'Dozent', dozent])
        kg_data.append([name, 'Veranstaltung von', chair])
    except Exception as e:
        print(e, 'bei Lecture:', lecture)

 36%|████████████████████████████▍                                                 | 1027/2820 [13:56<15:06,  1.98it/s]

list index out of range bei Lecture: https://campusonline.uni-bayreuth.de/ubto/wbLv.wbShowLVDetail?pStpSpNr=267718


 45%|██████████████████████████████████▋                                           | 1256/2820 [16:52<58:05,  2.23s/it]

list index out of range bei Lecture: https://campusonline.uni-bayreuth.de/ubto/wbLv.wbShowLVDetail?pStpSpNr=267899


 67%|███████████████████████████████████████████████████▉                          | 1879/2820 [29:27<16:43,  1.07s/it]

list index out of range bei Lecture: https://campusonline.uni-bayreuth.de/ubto/wbLv.wbShowLVDetail?pStpSpNr=267973


 70%|██████████████████████████████████████████████████████▊                       | 1980/2820 [31:04<15:37,  1.12s/it]

list index out of range bei Lecture: https://campusonline.uni-bayreuth.de/ubto/wbLv.wbShowLVDetail?pStpSpNr=267753


100%|██████████████████████████████████████████████████████████████████████████████| 2820/2820 [47:19<00:00,  1.01s/it]


In [28]:
kg_df = pd.DataFrame(kg_data, columns=['source', 'relation', 'target'])
team_df = team_df[team_df.source.str.len() <= 80]
final = kg_df.append(team_df)

# Knowledge Graph bereinigen
Da teils fehlerhafte Informationen im Knowledge Graph enthalten sind, werden diese entfernt. Ferner werden die zahlreichen Relationen mithilfe eines Wörterbuchs vereinheitlicht um Synonyme auf eine globale Bezeichnung zu bringen.

In [ ]:
final = final[~final['relation'].str.contains('@')]
final = final[~final['relation'].str.contains('Lebenslauf')]
final = final[~final['relation'].str.contains('Mehr')]
final = final[final['target'] != '']
final = final[~final['relation'].str.contains('Universite')]
final = final[~final['source'].str.len() < 3]
final = final[~final['target'].str.len() < 3]

In [29]:
for i in ['Universitätsstraße', 'Universitätsstr.', 'Universität', 'University', 'Die', 'Beginn', 'in', 'im',
          'auch', 'derzeit', '', 'Literatures', 'More', 'siehe', 'Public', 'B', 'Language', 'African', 'Erlangen',
          'Zur', 'Scientific', 'For', 'Curriculum', '▹', '.', 'Das', 'Im Ruhestand', 'im Ruhestand', 'Nähere', 
          'z.Zt. in Elternzeit', 'Online', 'Vorlage']:
    final = final[final['relation'] != i]
final = final.reset_index(drop = True)

In [30]:
relation_dict = {
'vorlesungsnummer' : 'Vorlesungsnummer',
'veranstaltungsart' : 'Veranstaltungsart',
'dozent' : 'Dozent',
'veranstaltung von' : 'Veranstaltung von',
'head of research group' : 'Lehrstuhlinhaber',
'telefon' : 'Telefon',
'phone' : 'Telefon',
'room' : 'Raum',
'raum' : 'Raum',
'gebäude' : 'Raum',
'sprechstunde' : 'Sprechstunde',
'consultation' : 'Sprechstunde',
'consultation-hour' : 'Sprechstunde',
'consultation-hours' : 'Sprechstunde',
'hours' : 'Sprechstunde',
'e-mail' : 'E-Mail',
'email' : 'E-Mail',
'zimmer' : 'Raum',
'juniorprofessor' : 'Lehrstuhlinhaber',
'juniorprofessorin' : 'Lehrstuhlinhaber',
'juniorprofessur' : 'Lehrstuhlinhaber',
'professur' : 'Lehrstuhlinhaber',
'lehrbeauftragterin' : 'Wissenschaftlicher Mitarbeiter',
'Building' : 'raum',
'office' : 'Sprechstunde',
'secretary' : 'Sekretariat',
'sekretärin' : 'Sekretariat',
'technician' : 'Techniker', 
'graduate student' : 'Studentischer Mitarbeiter',
'undergraduate student' : 'Studentischer Mitarbeiter',
'phd student' : 'Doktorand',
'teacher' : 'Wissenschaftlicher Mitarbeiter',
'professor' : 'Lehrstuhlinhaber',
'consulation' : 'Sprechstunde',
'group leader' : 'Lehrstuhlinhaber',
'academic councillor' : 'Akademischer Rat',
'academic counselor' : 'Akademischer Rat',
'secretariat' : 'Sekretariat',
'technical assistant' : 'Techniker',
'phd' : 'Doktorand',
'doktorandin' : 'Doktorand',
'doktorant' : 'Doktorand',
'doktorantin' : 'Doktorand',
'sprechzeit' : 'Sprechstunde',
'wissenschaftlicher mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'wissenschaftliche mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'wiss. mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'wiss. mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'studentische hilfskraft' : 'Studentischer Mitarbeiter',
'lehrstuhlinhaberin' : 'Lehrstuhlinhaber',
'sekretariat' : 'Sekretariat',
'tel' : 'Telefon',
'mail' : 'E-Mail',
'fax' : 'Fax',
'fox' : 'Fax',
'research assistant' : 'Wissenschaftlicher Mitarbeiter',
'master studentin' : 'Studentischer Mitarbeiter',
'master student' : 'Studentischer Mitarbeiter',
'akad. rat a.z' : 'Akademischer Rat',
'akad. rat. a.z.' : 'Akademischer Rat',
'chairholder' : 'Lehrstuhlinhaber',
'acad. director' : 'Akademischer Rat',
'staff scientist' : 'Wissenschaftlicher Mitarbeiter',
'ph.d. student' : 'Doktorand',
'ph d student' : 'Doktorand',
'ph d. student' : 'Doktorand',
'bachelor student' : 'Studentischer Mitarbeiter',
'bachelor studentin' : 'Studentischer Mitarbeiter',
'lehrkraft' : 'Wissenschaftlicher Mitarbeiter',
'physical engineer' : 'Wissenschaftlicher Mitarbeiter',
'vertretungsprofessur' : 'Vertretungsprofessur',
'doktorand' : 'Doktorand',
'sprechzeit' : 'Sprechstunde',
'sprechzeiten' : 'Sprechstunde',
'Sprechstunden' : 'Sprechstunde',
'akademische rätin' : 'Akademischer Rat',
'lehrstuhlinhaber' : 'Lehrstuhlinhaber',
'lehrstuhlinhaberin' : 'Lehrstuhlinhaber',
'wissenschaftliche hilfskraft' : 'Wissenschaftlicher Mitarbeiter',
'inhaberin' : 'Lehrstuhlinhaber',
'inhaber' : 'Lehrstuhlinhaber',
'professorin' : 'Lehrstuhlinhaber',
'adresse' : 'Raum',
'technische assistentin' : 'Wissenschaftlicher Mitarbeiter',
'technischer assistent' : 'Wissenschaftlicher Mitarbeiter',
'student assistant' : 'Studentischer Mitarbeiter',
'teamassistenz' : 'Wissenschaftlicher Mitarbeiter',
'academic council' : 'Akademischer Rat',
'lehrbeauftragte' : 'Lehrstuhlinhaber',
'lehrbeauftragter' : 'Lehrstuhlinhaber',
'projektmitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'projektmitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'büro' : 'Raum',
'building' : 'Raum',
'akademischer rat' : 'Akademischer Rat',
'akademische rätin' : 'Akademischer Rat',
'akademischer oberrat' : 'Akademischer Rat',
'akademische oberrätin' : 'Akademischer Rat',
'akademischer direktor' : 'Akademischer Direktor', 
'akademic director' : 'Akademischer Direktor', 
'alumni' : 'Alumni',
'bürozeiten' : 'Sprechstunde',
'laboratory assistant' : 'Wissenschaftlicher Mitarbeiter',
'post-doc' : 'Wissenschaftlicher Mitarbeiter',
'post doc' : 'Wissenschaftlicher Mitarbeiter',
'masterand' : 'Studentischer Mitarbeiter',
'masterandin' : 'Studentischer Mitarbeiter',
'bachelorand' : 'Studentischer Mitarbeiter',
'bachelorandin' : 'Studentischer Mitarbeiter',
'assistent' : 'Wissenschaftlicher Mitarbeiter',
'assistentin' : 'Wissenschaftlicher Mitarbeiter',
'student' : 'Studentischer Mitarbeiter',
'sprechstunde' : 'Sprechstunde',
'öffnungszeit' : 'Sprechstunde',
'öffnungszeiten' : 'Sprechstunde',
'mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'angestellter' : 'Wissenschaftlicher Mitarbeiter',
'habilitandin' : 'Habilitand',
'habilitand' : 'Habilitand',
'address' : 'Raum', 
'sprechstunden' : 'Sprechstunde',
'honorarprofessor' : 'Honorarprofessor',
'honorarprofessorin' : 'Honorarprofessor',
'lehrstuhlvertretung' : 'Lehrstuhlvertretung',
'mitarbeiter' : 'Wissenschaftlicher Mitarbeiter',
'mitarbeiterin' : 'Wissenschaftlicher Mitarbeiter',
'leiter' : 'Leiter',
'leiterin' : 'Leiter',
'austausch-doktorand' : 'Doktorand',
'bachelor-student' : 'Studentischer Mitarbeiter',
'bachelor-studentin' : 'Studentischer Mitarbeiter',
'master-student' : 'Studentischer Mitarbeiter',
'master-studentin' : 'Studentischer Mitarbeiter',
'postdoc' : 'Wissenschaftlicher Mitarbeiter',
'head of research' : 'Lehrstuhlinhaber',
'homepage' : 'Webseite',
'website' : 'Webseite',
'webseite' : 'Webseite',
'studentische hilfskräfte' : 'Studentischer Mitarbeiter',
'research scientist' : 'Wissenschaftlicher Mitarbeiter',
'lecturer' : 'Wissenschaftlicher Mitarbeiter',
'telefax' : 'Fax',
'assistant' : 'Wissenschaftlicher Mitarbeiter',
'privatdozent' : 'Privatdozent',
'privatdozentin' : 'Privatdozent',
'arbeitsgruppenleiter' : 'Arbeitsgruppenleiter',
'leitung' : 'Leitung',
'head' : 'Leitung',
'head of chair' : 'Lehrstuhlinhaber',
'wissenschaftliche' : 'Wissenschaftlicher Mitarbeiter',
'wissenschaftlicher' : 'Wissenschaftlicher Mitarbeiter',
'telefonsprechstunde' : 'Sprechstunde',
'studierendensekretariat' : 'Sekretariat',
'promovendin' : 'Doktorand',
'promovierende' : 'Doktorand',
'promovend' : 'Doktorand',
'promovierender' : 'Doktorand',
'fachleitung' : 'Leitung',
'techniker' : 'Techniker',
'technikerin' : 'Techniker',
'fachleiter' : 'Leitung',
'fachleiterin' : 'Leitung',
'lehrstuhlsekretärin' : 'Sekretariat',
'lektor' : 'Wissenschaftlicher Mitarbeiter',
'w1-juniorprofessorin' : 'Lehrstuhlinhaber',
'associate' : 'Wissenschaftlicher Mitarbeiter',
'studentische' : 'Studentischer Mitarbeiter',
'studiengangsmoderator' : 'Studiengangsmoderator',
'studiengangsmoderatorin' : 'Studiengangsmoderator',
'dekan' : 'Dekan',
'verwaltungsangestellte' : 'Sekretariat',
'technik' : 'Techniker',
'reinraumtechniker' : 'Techniker',
'emeritus' : 'Emeritus',
'principal investigator' : 'Wissenschaftlicher Mitarbeiter',
'studiengangskoordinatorin' : 'Studiengangskoordinator',
'studiengangskoordinator' : 'Studiengangskoordinator',
'oder' : 'E-Mail',
'gründungsdekan' : 'Dekan',
'geschäftsführer' : 'Geschäftsführer',
'appointments' : 'Sprechstunde',
'appointment' : 'Sprechstunde',
'scholar' : 'Wissenschaftlicher Mitarbeiter'
}

for i in relation_dict.copy():
    relation_dict[unicodedata.normalize('NFKD', i)] = relation_dict[i]
    
weekdays = ['montag', 'monday', 'dienstag', 'tuesday', 'mittwoch', 'wednesday', 'donnerstag', 'thursday',
            'freitag', 'friday', 'samstag', 'saturday', 'sonntag', 'sunday', 'mo', 'di', 'mi', 'do', 'fr', 'sa', 'so']

def check_for_wrong_relation_format(row):
    """Der typische Fehler, dass Informationen zwar korrekt, aber in der falschen Reihenfolge
    sind (relation wurde als Knoten erfasst), wird hiermit beseitigt."""
    relation = row['relation']
    target = row['target']
    relation = ''.join((char if char not in string.punctuation else ' ') for char in relation).split()
    if relation[0].lower() in weekdays:
        target = ' '.join(relation) + target
        relation = 'Sprechstunde'
        return True, relation, target
    if len(relation) > 1:
        relation_first = relation_dict.get(relation[0].lower(), 'not found')
        if (relation_first == 'Telefon') or (relation_first == 'Fax') or (relation_first == 'Raum') or (relation_first == 'E-Mail') or (relation_first == 'Sprechstunde'):
            if target.isalpha():
                target = ' '.join(relation[1:]) + target
                relation = relation_first
            else:
                target = ' '.join(relation[1:])
                relation = relation_first
            return True, relation, target
    return False, relation, target

for i in range(len(final)):
    to_change, relation, target = check_for_wrong_relation_format(final.iloc[i])
    if to_change:
        final.loc[i, 'relation'] = relation
        final.loc[i, 'target'] = target

not_found = []
def clean_relations(relation):
    """Vereinheitlicht die Relationen regelbasiert."""
    global not_found
    relation = relation.lower()
    right_relation = relation_dict.get(relation, 'not found')
    if right_relation == 'not found':
        right_relation = check_relation_permutations(relation)
    if right_relation == 'not found':
        not_found.append(relation)
        return relation
    else:
        return right_relation
    

def check_relation_permutations(relation):
    """Ermittelt alle Permutationen der Wörter einer Relation."""
    relation = relation.split(' ')
    unicode_relation = [unicodedata.normalize('NFKD', x) for x in relation]
    relation_clean = [x.strip(string.punctuation) for x in unicode_relation]
    relation = [x.strip() for x in relation_clean]
    if len(relation) > 10:
        perm = permutations(relation, 1)
        for j in perm:
            perm_joined = j[0]
            if perm_joined in relation_dict:
                return relation_dict[perm_joined]
            else:
                return 'not found'
    for i in range(len(relation), 0, -1):
        perm = permutations(relation, i)
        for j in perm:
            perm_joined = ' '.join(j)
            if perm_joined in relation_dict:
                return relation_dict[perm_joined]
    return 'not found'

In [35]:
final['relation_cleaned'] = final['relation'].apply(clean_relations)
final = final[['source' , 'relation_cleaned', 'target']]
final.columns = ['source', 'relation', 'target']
final = final[~final['relation'].isin(not_found)]

professoren = final[final['relation'] == 'Lehrstuhlinhaber'].drop_duplicates()
chair_dict = dict(zip(professoren.source.to_list(), professoren.target.to_list()))

for key in chair_dict.keys():
    try:
        veranstaltungen = final[final['target'] == key].source.to_list()
        chair_to_change = final[(final['source'].isin(veranstaltungen)) & (final['relation'] == 'Veranstaltung von')].target.value_counts().index[0]
        final.loc[final['target'] == chair_to_change, 'target'] = chair_dict[key]
    except Exception as e:
        print(e, 'at', key)

# Mensa-Speisepläne Scrapen
Um Informationen über das Menü in der Mensa zur Verfügung zu stellen, wird nun die Webseite des Studentenwerks gescraped.

In [66]:
iso = datetime.datetime.today().isocalendar()
d = str(iso[0]) + '-W' + str(iso[1]-1)
dates = []
for i in range(0,7):
    r = datetime.datetime.strptime(d + '-' + str(i), "%Y-W%W-%w")
    dates.append(str(r).split()[0])
dates.sort()
dayname = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']

In [73]:
meal_list = []
for date in range(len(dates)):
    base_url = 'https://www.studentenwerk-oberfranken.de/essen/speiseplaene/bayreuth/frischraum/'
    page = requests.get(base_url + dates[date] + '.html')
    soup = BeautifulSoup(page.content, 'html.parser')
    temp = soup.find_all('div', class_ = "tx-bwrkspeiseplan__hauptgerichte")
    meals = []
    if len(temp) < 2:
        continue
    for i in temp[1].find_all('td')[::3]:
        raw = i.text.split('\n')[0].strip()
        clean = raw.split('(')[0].strip()
        meals.append(clean)
    meal_list.append(meals[:-2])
kg_format = []
for j in range(len(meal_list)):
    kg_format.append([dayname[j], 'Mensa', str(meal_list[j]).strip('[]').strip("''")])

In [75]:
final = final.append(pd.DataFrame(kg_format, columns=['source', 'relation', 'target']))

# Klausurenplan
Um Informationen über Klausurentermine etc. bereitzustellen, wird hier ein gegebener Klausurenplan in den Knowledge Graph eingefügt.

In [171]:
exams = []
f = open('Klausuren.txt', 'r', encoding='utf-8')
line = f.readline()
while line:
    exams.append(line.rstrip())
    line = f.readline()
f.close()

In [172]:
exams_ordered = []
last = 0
for i in range(len(exams)):
    if '|' in exams[i]:
        exams_ordered.append(exams[last:i])
        last = i + 1

In [173]:
info_ordered = []
for exam_chunk in exams_ordered:
    date = exam_chunk.pop(0)
    last = 0
    for i in range(len(exam_chunk)):
        if ';' in exam_chunk[i]:
            temp = [date]
            temp.extend(exam_chunk[last:i+1])
            last = i + 1
            info_ordered.append(temp)

In [174]:
kg_format = []
for exam in info_ordered:
    exam = [x.strip(';') for x in exam]
    date = exam[0]
    name = exam[1]
    kg_format.append([name, 'Klausur', date + ' ' + ' '.join(exam[2:])])

In [35]:
final = final.append(pd.DataFrame(kg_format, columns = ['source', 'relation', 'target']))

# E-Learning Links Scrapen
Um schnell den Link zu der E-Learning Page einer Veranstaltung zu bekommen, wird nun das E-Learning der Uni Bayreuth gescraped

In [32]:
def scrape_elearning(urls):
    """Scraped für eine Liste von URLs die Elearning Links zu Kursen."""
    new_urls = []
    kg_data = []
    for url in tqdm(urls):
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        temp = soup.find_all('div', class_ = 'subcategories')
        if len(temp) == 0:
            temp = soup.find_all('div', class_ = 'content')
            courses = temp[0].find_all('h3', class_='coursename')
            for course in courses:
                info = course.find_all('a')[0]
                new_url = info['href']
                coursename = info.text
                coursename = clean_course_name(coursename)
                new_urls.append(new_url)
                kg_data.append([coursename, 'e-Learning', new_url])
        else:
            for a in temp[0].find_all('a'):
                kg_data.append([clean_course_name(a.text), 'e-Learning', a['href']])
                new_urls.append(a['href'])
    return len(new_urls) > 0, new_urls, kg_data

def clean_course_name(course_name):
    """Bereinigt den Kursnamen um Strings wie z.B. SoSe 2020."""
    common_expressions = ['SS2020', 'SoSe2020', 'SS20',
                         'SS2020', 'SS 2020', 'SS 20',
                         'SoSe 2020', 'SoSe 20', 'Sommersemester 20',
                         'Sommersemester 2020', '2020', '20', 'SS',
                         'SoSe', 'Sommersemester'] 
    course_name = re.sub("[\(\[].*?[\)\]]", "", course_name)
    course_name = ' '.join([x for x in course_name.split() if x not in common_expressions]).strip()
    return course_name
    

In [33]:
keep_going = True
new_urls = ['https://elearning.uni-bayreuth.de/course/index.php?categoryid=4140']
kg_data_global = []
while keep_going:
    keep_going, new_urls, kg_data = scrape_elearning(new_urls)
    kg_data_global.extend(kg_data)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:10<00:00,  2.23it/s]


In [ ]:
final = final.append(pd.DataFrame(kg_data_global, columns = ['source', 'relation', 'target']))
final.to_csv('/home/ec2-user/data/knowledge_graph_relations_cleaned.csv', index = False, sep = '|')